In [ ]:
!nvidia-smi

Fri Mar 26 03:51:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    50W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
cd /content/drive/MyDrive/workspace/lotte/

/content/drive/MyDrive/workspace/lotte


In [ ]:
!unzip ./dataset/LPD_competition.zip -d /content

In [ ]:
!pip install efficientnet_pytorch
!pip install pytorch_pretrained_vit

### 라이브러리 import

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from efficientnet_pytorch import EfficientNet
from pytorch_pretrained_vit import ViT
from tqdm.auto import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import models
from glob import glob
import cv2
from PIL import Image
import torch.nn.functional as F

### Config Setting

In [ ]:
#CONFIG
torch.manual_seed(777)
BATCH_SIZE=100
#DEVICE
print(f'PyTorch Version : [{torch.__version__}]')
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Device : [{device}]')

PyTorch Version : [1.8.0+cu101]
Device : [cuda:0]


### Custom Datasets

In [ ]:
class LotteDataset(Dataset):
  def __init__(self, data_root, train_mode):
    super(LotteDataset, self).__init__()
    self.train_mode=train_mode

    if self.train_mode==False:
      self.img_list = glob(os.path.join(data_root, '*.jpg'))
      self.img_list.sort(key=lambda x:int(x.split('/')[3][:-4]))
    else:
      self.img_list = glob(os.path.join(data_root, '*/*.jpg'))
      self.train_y=[]
      for img_path in self.img_list:
        self.train_y.append(int(img_path.split('/')[3]))
        
    self.len = len(self.img_list)

  def __getitem__(self, index):
    img_path = self.img_list[index]
    if self.train_mode:
      label=int(img_path.split('/')[3])
    # Image Loading
    img = Image.open(img_path)

    if self.train_mode:
      return img,label
    else:
      return img

  def __len__(self):
    return self.len

In [ ]:
class MapTransform(Dataset):
    def __init__(self, dataset, transform, train_mode):
        self.dataset = dataset
        self.transform=transform
        self.train_mode=train_mode

    def __getitem__(self, index):
        if self.train_mode:
          return self.transform(self.dataset[index][0]), self.dataset[index][1]
        else:
          return self.transform(self.dataset[index])

    def __len__(self):
        return len(self.dataset)

In [ ]:
test_transforms=transforms.Compose([transforms.ToTensor(),
                                    transforms.Resize((224,224)),
                                    transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])

test_data=LotteDataset('/content/test',train_mode=False)

trans_test_data=MapTransform(test_data,test_transforms,train_mode=False)
test_iter=DataLoader(trans_test_data,batch_size=BATCH_SIZE,shuffle=False,num_workers=2)

### K-Fold Ensemble 기반 다양한 아키텍쳐 Model Ensemble

In [ ]:
class KfoldModel(nn.Module):
  def __init__(self,Fold_1,Fold_2,Fold_3,Fold_4,sharp=True):
    super(KfoldModel,self).__init__()
    self.Fold_1=Fold_1
    self.Fold_2=Fold_2
    self.Fold_3=Fold_3
    self.Fold_4=Fold_4
    self.F_softmax=nn.Softmax(dim=1)

    self.sharp=sharp
    self.T=0.5

  def forward(self,x):
    a=self.Fold_1(x.clone().detach())
    b=self.Fold_2(x.clone().detach())
    c=self.Fold_3(x.clone().detach())
    d=self.Fold_4(x.clone().detach())

    a=self.F_softmax(a)
    b=self.F_softmax(b)
    c=self.F_softmax(c)
    d=self.F_softmax(d)
    if self.sharp:
      return (a**self.T+b**self.T+c**self.T+d**self.T)/4.
    else:
      return (a+b+c+d)/4.

### Resnext50

In [ ]:
#/content/drive/MyDrive/workspace/lotte/checkpoint_resnext50/0_Resnext50_epoch_27.tar
path1='./checkpoint_resnext50/0_Resnext50_epoch_27.tar'
path2='./checkpoint_resnext50/1_Resnext50_epoch_29.tar'
path3='./checkpoint_resnext50/2_Resnext50_epoch_27.tar'
path4='./checkpoint_resnext50/3_Resnext50_epoch_24.tar'

Resnext50_Fold_1=models.resnext50_32x4d(pretrained=False)
checkpoint=torch.load(path1)
Resnext50_Fold_1.load_state_dict(checkpoint['model_state_dict'])
Resnext50_Fold_1.eval()

Resnext50_Fold_2=models.resnext50_32x4d(pretrained=False)
checkpoint=torch.load(path2)
Resnext50_Fold_2.load_state_dict(checkpoint['model_state_dict'])
Resnext50_Fold_2.eval()

Resnext50_Fold_3=models.resnext50_32x4d(pretrained=False)
checkpoint=torch.load(path3)
Resnext50_Fold_3.load_state_dict(checkpoint['model_state_dict'])
Resnext50_Fold_3.eval()

Resnext50_Fold_4=models.resnext50_32x4d(pretrained=False)
checkpoint=torch.load(path4)
Resnext50_Fold_4.load_state_dict(checkpoint['model_state_dict'])
Resnext50_Fold_4.eval()

Model_Resnext50=KfoldModel(Resnext50_Fold_1,Resnext50_Fold_2,Resnext50_Fold_3,Resnext50_Fold_4,sharp=True)
Model_Resnext50.eval()

### EfficientNet b04

In [ ]:
#/content/drive/MyDrive/workspace/lotte/checkpoint_efficient04/0_efficient04_epoch_26.tar
path1='./checkpoint_efficient04/0_efficient04_epoch_26.tar'
path2='./checkpoint_efficient04/1_efficient04_epoch_24.tar'
path3='./checkpoint_efficient04/2_efficient04_epoch_29.tar'
path4='./checkpoint_efficient04/3_efficient04_epoch_23.tar'

Efficient04_Fold_1=EfficientNet.from_name('efficientnet-b4')
checkpoint=torch.load(path1)
Efficient04_Fold_1.load_state_dict(checkpoint['model_state_dict'])
Efficient04_Fold_1.eval()

Efficient04_Fold_2=EfficientNet.from_name('efficientnet-b4')
checkpoint=torch.load(path2)
Efficient04_Fold_2.load_state_dict(checkpoint['model_state_dict'])
Efficient04_Fold_2.eval()

Efficient04_Fold_3=EfficientNet.from_name('efficientnet-b4')
checkpoint=torch.load(path3)
Efficient04_Fold_3.load_state_dict(checkpoint['model_state_dict'])
Efficient04_Fold_3.eval()

Efficient04_Fold_4=EfficientNet.from_name('efficientnet-b4')
checkpoint=torch.load(path4)
Efficient04_Fold_4.load_state_dict(checkpoint['model_state_dict'])
Efficient04_Fold_4.eval()

Model_Efficient04=KfoldModel(Efficient04_Fold_1,Efficient04_Fold_2,Efficient04_Fold_3,Efficient04_Fold_4,sharp=True)
Model_Efficient04.eval()

### Resnext101

In [ ]:
#/content/drive/MyDrive/workspace/lotte/checkpoint_resnext101/0_Resnext_epoch_29.tar
path1='./checkpoint_resnext101/0_Resnext_epoch_29.tar'
path2='./checkpoint_resnext101/1_Resnext_epoch_28.tar'
path3='./checkpoint_resnext101/2_Resnext_epoch_29.tar'
path4='./checkpoint_resnext101/3_Resnext_epoch_27.tar'

Resnext101_Fold_1=models.resnext101_32x8d(pretrained=False)
checkpoint=torch.load(path1)
Resnext101_Fold_1.load_state_dict(checkpoint['model_state_dict'])
Resnext101_Fold_1.eval()

Resnext101_Fold_2=models.resnext101_32x8d(pretrained=False)
checkpoint=torch.load(path2)
Resnext101_Fold_2.load_state_dict(checkpoint['model_state_dict'])
Resnext101_Fold_2.eval()

Resnext101_Fold_3=models.resnext101_32x8d(pretrained=False)
checkpoint=torch.load(path3)
Resnext101_Fold_3.load_state_dict(checkpoint['model_state_dict'])
Resnext101_Fold_3.eval()

Resnext101_Fold_4=models.resnext101_32x8d(pretrained=False)
checkpoint=torch.load(path4)
Resnext101_Fold_4.load_state_dict(checkpoint['model_state_dict'])
Resnext101_Fold_4.eval()

Model_Resnext101=KfoldModel(Resnext101_Fold_1,Resnext101_Fold_2,Resnext101_Fold_3,Resnext101_Fold_4,sharp=True)
Model_Resnext101.eval()

### ViT

In [ ]:
#/content/drive/MyDrive/workspace/lotte/checkpoint_vit/ViT2_epoch_0.tar
path='./checkpoint_vit/ViT2_epoch_0.tar'

Model_ViT = ViT('B_16_imagenet1k', pretrained=False,image_size=224)
checkpoint=torch.load(path)
Model_ViT.load_state_dict(checkpoint['model_state_dict'])
Model_ViT.eval()

## Inference

In [ ]:
with torch.no_grad():
  Model_Resnext50.eval(), Model_Resnext50.to(device)
  Model_Efficient04.eval(), Model_Efficient04.to(device)
  Model_Resnext101.eval(), Model_Resnext101.to(device)
  Model_ViT.eval(), Model_ViT.to(device)
  pred_label=[]
  print("Ensemble Inference....\n")
  for imgs in tqdm(iter(test_iter)):
    imgs=imgs.to(device)
    A_pred=Model_Resnext50(imgs)
    B_pred=Model_Efficient04(imgs)
    C_pred=Model_Resnext101(imgs)
    D_pred=F.softmax(Model_ViT(imgs),dim=1)

    model_pred=(A_pred**0.5+B_pred**0.5+C_pred**0.5+D_pred**0.5)/4.
    _, y_pred=torch.max(model_pred.data,1)
    pred_label.extend(y_pred.tolist())

submission = pd.read_csv('./dataset/sample.csv', encoding = 'utf-8')
submission['prediction'] = pred_label
submission.to_csv('./submission_softvoing.csv', index = False)

Ensemble Inference....

